In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
# load local version of gallifrey, before pip installed version, for debugging
import pathlib
import sys
import os

sys.path.append(str(pathlib.Path(os.getcwd()).parent.joinpath("src")))

In [3]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import yt
from matplotlib.ticker import ScalarFormatter

from gallifrey.analysis import count_planets
from gallifrey.model import Model
from gallifrey.utilities.dataframe import (
    rename_labels,
    rename_entries,
    aggregated_dataframe,
)
from gallifrey.utilities.math import calculate_pca
from gallifrey.visualization.visualization_utils import (
    FigureProcessor,
    adjust_legend,
    get_palette,
    ridgeplot,
    set_plot_defaults,
)

## DATA SETUP

In [4]:
snapshot = 127
resolution = 8192
sim_id = "37_11"
num_embryos = 50
host_star_masses = 1
planet_categories = [
    "Earth",
    "Super-Earth",
    "Neptunian",
    "Sub-Giant",
    "Giant",
]

In [5]:
model = Model(
    snapshot=snapshot,
    resolution=resolution,
    sim_id=sim_id,
)
ds = model.ds
mw = model.halo

yt : [INFO     ] 2023-11-24 11:45:07,786 Calculating time from 1.000e+00 to be 4.318e+17 seconds
yt : [INFO     ] 2023-11-24 11:45:07,941 Parameters: current_time              = 4.318436660023809e+17 s
yt : [INFO     ] 2023-11-24 11:45:07,943 Parameters: domain_dimensions         = [1 1 1]
yt : [INFO     ] 2023-11-24 11:45:07,944 Parameters: domain_left_edge          = [0. 0. 0.]
yt : [INFO     ] 2023-11-24 11:45:07,945 Parameters: domain_right_edge         = [100. 100. 100.]
yt : [INFO     ] 2023-11-24 11:45:07,946 Parameters: cosmological_simulation   = 1
yt : [INFO     ] 2023-11-24 11:45:07,947 Parameters: current_redshift          = 4.440892098500626e-16
yt : [INFO     ] 2023-11-24 11:45:07,948 Parameters: omega_lambda              = 0.682
yt : [INFO     ] 2023-11-24 11:45:07,949 Parameters: omega_matter              = 0.318
yt : [INFO     ] 2023-11-24 11:45:07,949 Parameters: omega_radiation           = 0.0
yt : [INFO     ] 2023-11-24 11:45:07,950 Parameters: hubble_constant      

## PLOT SETUP

In [6]:
set_plot_defaults()

figure_subdirectory = f"MilkyWayAnalogue/{model.resolution}/{model.sim_id}/{model.snapshot}/Component_populations"
save = True

## CREATE DATASOURCE

In [7]:
radius = 0.2 * mw.virial_radius()
masses = [1]  # (0.1, 0.5, 1)
components = ["Bulge", "Thin Disk", "Thick Disk", "Halo"]

sphere = mw.sphere(radius=radius)

## PLANET AGES PER COMPONENT

In [8]:
data = aggregated_dataframe(
    ["bulge_stars", "thin_disk_stars", "thick_disk_stars", "halo_stars"],
    ["stellar_age", *planet_categories],
    data_source=sphere,
    type_name="Component",
    custom_units={"stellar_age": "Gyr"},
)
data = rename_entries(data)

In [ ]:
data.groupby("Component").apply(lambda x: np.average(x.wt, weights=x.value))

stellar_age                                          \
                       count      mean       std       min       25%   
Component                                                              
bulge_stars         707269.0  8.613635  2.473027  0.021317  7.058386   
halo_stars          188691.0  9.687081  2.063464  0.030658  8.619065   
thick_disk_stars    154563.0  8.488042  2.022401  0.026203  7.078220   
thin_disk_stars    1498647.0  5.998274  2.440044  0.020019  4.637591   

                                                  
                       50%        75%        max  
Component                                         
bulge_stars       9.049882  10.428129  13.592348  
halo_stars        9.951314  11.110012  13.536657  
thick_disk_stars  8.674826   9.868500  13.539622  
thin_disk_stars   6.089130   7.584162  13.561366

## OCCURENCE RATES PER COMPONENT

In [8]:
occurence_rate_long_format = {}
for m in masses:
    occurence_rate_long_format[m] = count_planets(
        model,
        lambda: mw.sphere(radius=radius),
        planet_categories,
        normalize_by="planet_hosting_number",
        model_config={"ngpps_star_masses": m},
        long_format=True,
        value_name="Occurence Rate",
    )


      CONFIG: Setting ngpps_star_masses to 0.1.


      TIMER: Block 'Adding Planets...' took 3.42 seconds.


      CONFIG: Setting ngpps_star_masses to 0.5.


      TIMER: Block 'Adding Planets...' took 3.43 seconds.


      CONFIG: Setting ngpps_star_masses to 1.


      TIMER: Block 'Adding Planets...' took 3.22 seconds.



In [ ]:
def barplot(occurence_rate_long_format, host_star_mass):
    plt.figure()
    with sns.axes_style("ticks"):
        occurence_rate_barplot = sns.barplot(
            data=occurence_rate_long_format,
            x="Planet Type",
            y="Occurence Rate",
            hue="Component",
            hue_order=components,
            log=True,
            palette=get_palette(),
        )

    FigureProcessor(occurence_rate_barplot.figure).save(
        f"occurence_rates_with_host_star_mass_={host_star_mass}.pdf",
        sub_directory=figure_subdirectory,
        save=save,
    )
    return occurence_rate_barplot


for m in masses:
    plot = barplot(occurence_rate_long_format[m], host_star_mass=m)